In [31]:
from urllib.request import urlopen
from urllib.parse import quote_plus # 아스키 코드로 변환시켜준다.
import urllib.request
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException
from requests.exceptions import ConnectionError
import unicodedata
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import os
import urllib.request        
import filecmp
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from glob import glob

In [2]:
def isKorean(text):
    hangul = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')  
    result = hangul.findall(text)
    return len(result)

In [79]:
yyyy == None

NameError: name 'yyyy' is not defined

In [85]:
def upload_date_check():
    html = driver.page_source
    soup = BeautifulSoup(html)
    # driver.find_element_by_xpath('.//time [@class = "_aaqe"]').text# datetime
    # time.sleep(1)
    

    for idx, i in enumerate(soup.findAll('time')):
        if i.has_attr('datetime'):
            TIME = i['datetime']

            p = re.search('(\d+)-(\d+)-(\d+)', TIME)
            if p:
                yyyy, mm, dd = [int(x) for x in p.groups()]
                print(datetime.datetime.now().date() - datetime.date(yyyy, mm, dd), end='\r')
                return yyyy, mm, dd
        
    return False
    # return datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)
    

In [20]:
def crawl_influencers():
    driver.find_element_by_partial_link_text("팔로우").click()
    #driver.find_element_by_css_selector('a').click()
    time.sleep(5)

    SCROLL_PAUSE = 1  # Pause to allow loading of content
    driver.execute_script("followingbox = document.getElementsByClassName('_aano')[0];")
    last_height = driver.execute_script("return followingbox.scrollHeight;")

    # We need to scroll the following modal to ensure that all following are loaded
    while True:
        driver.execute_script("followingbox.scrollTo(0, followingbox.scrollHeight);")

        # Wait for page to load
        time.sleep(SCROLL_PAUSE)

        # Calculate new scrollHeight and compare with the previous
        new_height = driver.execute_script("return followingbox.scrollHeight;")
        if new_height == last_height:
            time.sleep(3)
            if new_height == last_height:
                break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    followings = soup.findAll('a',['FPmhX','notranslate','_0imsa'])
    followings_text = []
    for following in followings:
        followings_text.append(following.get_text())
            
    rest= requests.get(url)
    soup =BeautifulSoup(rest.content, 'html.parser')

    summary_content = soup.find('meta', {'name': 'description'})['content']
    follower_count = summary_content.split(' ')[0]
    # print("팔로워 수: "+str(follower_count))
    following_count = summary_content.split(' ')[2]
    # print("팔로잉 수: "+str(following_count))
    post_count = summary_content.split(' ')[4]
    # print("게시물 수: "+str(post_count))
    
    # print("팔로잉 수: " + str(len(followings_text)-count))
    # following_count = len(followings_text)-count
    
    close_following_bttn = driver.find_element_by_class_name('_ac7b._ac7d')
    close_following_bttn.click()

    influencer_following = [ x.split('인증됨')[0] for x in followings_text if len(x.split('인증됨'))==2 ]
    
    return influencer_following, follower_count

In [21]:
def image_save(SRC, PATH, UPLOAD_DATE):
    yyyy, mm, dd = UPLOAD_DATE
    exists_files = glob(f'{download_directory}/{datetime.date(yyyy, mm, dd)}*')
    with open(f'{PATH}/{datetime.date(yyyy, mm, dd)}_{len(exists_files)+1}.jpg', 'wb') as handler: 
        handler.write(SRC)

In [22]:
personal_info = pd.read_csv('personal_info.csv')
ID, PW = personal_info['ID'], personal_info['PW']

In [102]:
influencers = pd.read_csv('new_influencer.csv', index_col=0)
# influencers = [x.strip() for x in influencers['influencer'].values]
exists_df = influencers.index.isin(os.listdir('./img'))
influencers = influencers[~exists_df]

In [96]:
influencers

,instagram
influencer,
michaelkors,https://www.instagram.com/michaelkors/
tamumcpherson,https://www.instagram.com/tamumcpherson/
phillip_hkkim,https://www.instagram.com/phillip_hkkim/
alicefunk9093,https://www.instagram.com/alicefunk9093/
parkhyoshin.official,https://www.instagram.com/parkhyoshin.official/
...,...
evachen212,https://www.instagram.com/evachen212/
noramilch,https://www.instagram.com/noramilch/
sincerelyjules,https://www.instagram.com/sincerelyjules/


In [25]:
baseUrl = 'https://www.instagram.com/'
plusUrl = influencers.index[0]# 'jin_a_nana'# influencers['influencer'][9].strip()
url = baseUrl + quote_plus(plusUrl)

print(url)

https://www.instagram.com/lewishamilton


In [90]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(baseUrl) # 드라이버를 띄운다. (크롬 웹 페이지를 연다.)
driver.implicitly_wait(5)

driver.find_element_by_name('username').send_keys(ID)
driver.implicitly_wait(5)
elem_pw = driver.find_element_by_name('password') # .submit()
elem_pw.send_keys(PW)
driver.implicitly_wait(5)
elem_pw.submit()
driver.implicitly_wait(5)
time.sleep(5)

In [91]:
influencers.index[0]

'michaelkors'

In [97]:
baseUrl = 'https://www.instagram.com/'
plusUrl = influencers.index[0]# 'jin_a_nana'# influencers['influencer'][9].strip()
url = baseUrl + quote_plus(plusUrl)
download_directory = os.path.join('./img', plusUrl)

for plusUrl in influencers.index:
    url = baseUrl + quote_plus(plusUrl)
    print(url)

https://www.instagram.com/michaelkors
https://www.instagram.com/tamumcpherson
https://www.instagram.com/phillip_hkkim
https://www.instagram.com/alicefunk9093
https://www.instagram.com/parkhyoshin.official
https://www.instagram.com/jh_han
https://www.instagram.com/happpy_yein
https://www.instagram.com/yuetyuetxx
https://www.instagram.com/yoonjujang
https://www.instagram.com/mariagraziachiuri
https://www.instagram.com/baritong
https://www.instagram.com/harpersbazaarkorea
https://www.instagram.com/_yunjeekim_
https://www.instagram.com/leojmakeup
https://www.instagram.com/y1003_grace
https://www.instagram.com/celine
https://www.instagram.com/sedeukoh
https://www.instagram.com/chefsamkim
https://www.instagram.com/lunadelizia
https://www.instagram.com/hyojoo.p
https://www.instagram.com/prequelapp
https://www.instagram.com/austinkangg
https://www.instagram.com/every__nn
https://www.instagram.com/xinsooo
https://www.instagram.com/kyo1122
https://www.instagram.com/kangyewon0315
https://www.inst

In [98]:
influencers.index[0]

'michaelkors'

In [99]:
# influencers = influencers.drop(influencers.index[0])

# influencers.to_csv('new_influencer.csv')

In [100]:
influencers

,instagram
influencer,
michaelkors,https://www.instagram.com/michaelkors/
tamumcpherson,https://www.instagram.com/tamumcpherson/
phillip_hkkim,https://www.instagram.com/phillip_hkkim/
alicefunk9093,https://www.instagram.com/alicefunk9093/
parkhyoshin.official,https://www.instagram.com/parkhyoshin.official/
...,...
evachen212,https://www.instagram.com/evachen212/
noramilch,https://www.instagram.com/noramilch/
sincerelyjules,https://www.instagram.com/sincerelyjules/


In [115]:
influencers = pd.read_csv('new_influencer.csv', index_col=0)
influencers

,instagram
influencer,
kieunse,https://www.instagram.com/kieunse/
nayoungkeem,https://www.instagram.com/nayoungkeem/
ch_amii,https://www.instagram.com/ch_amii/
gyoe_re,https://www.instagram.com/gyoe_re/
doflwl,https://www.instagram.com/doflwl/
nugayoung,https://www.instagram.com/nugayoung/
hehehe0,https://www.instagram.com/hehehe0/
jmyoungl,https://www.instagram.com/jmyoungl/
3.48kg,https://www.instagram.com/3.48kg/


In [134]:
influencers = pd.read_csv('new_influencer.csv', index_col=0)
exists_df = influencers.index.isin(os.listdir('./img'))

influencers = influencers[~exists_df]
influencers

,instagram
influencer,
michaelkors,https://www.instagram.com/michaelkors/
tamumcpherson,https://www.instagram.com/tamumcpherson/
jh_han,https://www.instagram.com/jh_han/
yuetyuetxx,https://www.instagram.com/yuetyuetxx/
yoonjujang,https://www.instagram.com/yoonjujang/
...,...
evachen212,https://www.instagram.com/evachen212/
noramilch,https://www.instagram.com/noramilch/
sincerelyjules,https://www.instagram.com/sincerelyjules/


In [141]:
influencers.drop(influencers.index[0])

,instagram
influencer,
tamumcpherson,https://www.instagram.com/tamumcpherson/
jh_han,https://www.instagram.com/jh_han/
yuetyuetxx,https://www.instagram.com/yuetyuetxx/
yoonjujang,https://www.instagram.com/yoonjujang/
mariagraziachiuri,https://www.instagram.com/mariagraziachiuri/
...,...
evachen212,https://www.instagram.com/evachen212/
noramilch,https://www.instagram.com/noramilch/
sincerelyjules,https://www.instagram.com/sincerelyjules/


In [149]:
pd.read_csv('new_influencer.csv', index_col=0)

,instagram
influencer,
kieunse,https://www.instagram.com/kieunse/
nayoungkeem,https://www.instagram.com/nayoungkeem/
ch_amii,https://www.instagram.com/ch_amii/
gyoe_re,https://www.instagram.com/gyoe_re/
doflwl,https://www.instagram.com/doflwl/
nugayoung,https://www.instagram.com/nugayoung/
hehehe0,https://www.instagram.com/hehehe0/
jmyoungl,https://www.instagram.com/jmyoungl/
3.48kg,https://www.instagram.com/3.48kg/


In [148]:
influencers# .index[0]

,instagram
influencer,


In [ ]:
influencers

In [ ]:
# for plusUrl in influencers[len(os.listdir(r'C:\Users\anapl\CDAL\fashion\instagram-crawler\img')):]:
while True:    
    influencers = pd.read_csv('new_influencer.csv', index_col=0)
    exists_df = influencers.index.isin(os.listdir('./img'))
    influencers = influencers[~exists_df]    
    plusUrl = influencers.index[0] # 'happpy_yein'
    
    url = baseUrl + quote_plus(plusUrl)
    
    driver.get(url)
    time.sleep(3)
    
    rest = requests.get(url)
    soup = BeautifulSoup(rest.content, 'html.parser')
    
    if soup.find('meta', {'name': 'description'}) == None:
        print("\nSOUP NO FOUND")
        influencers = influencers.drop(influencers.index[0])
        influencers.to_csv('new_influencer.csv')
        continue
    
    # 이름 확인
    try:
        NAME = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/section/main/div/header/section/div[3]/span').text
        NAME = unicodedata.normalize('NFC', NAME)
    except NoSuchElementException:
        print(f"\nNO NAME {len(influencers)}")
        influencers = influencers.drop(influencers.index[0])
        influencers.to_csv('new_influencer.csv')
        continue
    
    if isKorean(NAME):
        pass
    else:
        print(f"\nNO KOREAN {len(influencers)}")
        influencers = influencers.drop(influencers.index[0])
        influencers.to_csv('new_influencer.csv')
        continue
        
    summary_content = soup.find('meta', {'name': 'description'})['content']
    follower_count = summary_content.split(' ')[0]
    # print("팔로워 수: "+str(follower_count))
    following_count = summary_content.split(' ')[2]
    # print("팔로잉 수: "+str(following_count))
    post_count = summary_content.split(' ')[4]
    # print("게시물 수: "+str(post_count))
    
    time.sleep(1)
    try:
        influencer_following, follower_count = crawl_influencers()
        insta_link = ['https://www.instagram.com/'+x+'/' for x in influencer_following]
        df = pd.DataFrame([influencer_following, insta_link], index=['influencer', 'instagram']).T
        df_inf = pd.read_csv('new_influencer.csv')
        new_df = pd.concat([df_inf, df]).reset_index(drop=True)
        new_df.to_csv("new_influencer.csv", index=False)
    except NoSuchElementException:
        print(f"\nNO FOLLOWING {len(influencers)}")
    
    body = driver.find_element_by_css_selector('body')

    download_directory = os.path.join('./img', plusUrl)

    html = driver.page_source
    soup = BeautifulSoup(html)
        
    if not os.path.isdir('./img'):
        os.mkdir('./img')

    if os.path.isdir(os.path.join('./img', plusUrl)):
        print(f"{plusUrl} Directory Exists")
        if len(os.listdir(os.path.join('./img', plusUrl))) != 0:
            continue

    else:
        os.mkdir(os.path.join('./img', plusUrl))
        print(f"{plusUrl} Make Directory")
    
        
    # 첫번째 이미지 클릭
    btn_images = driver.find_elements_by_class_name('_aabd._aa8k._aanf')
    # print(len(btn_images))
    # btn[-1].click()

    btn = btn_images[0]
    btn.click()
    time.sleep(2)

    downloaded_image_list = []
    global_stack = 0
    while True:
        image_list = []
        exists = os.listdir(download_directory)
        while True:
            if upload_date_check() != False:
                yyyy, mm, dd = upload_date_check()
                upload_date = (yyyy, mm, dd)
                time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)
            else:
                try:
                    btn_next =  driver.find_element_by_class_name('_9zm2')
                    btn_next.click()
                except NoSuchElementException:
                    try:
                        next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                        next_post.click()
                    except NoSuchElementException:
                        pass
                    finally:
                        break

            if time_delta > datetime.timedelta(days=90):
                break

            for x in driver.find_elements_by_xpath('//div/img')[-3:]:
                if x.get_property('alt').startswith('Photo'): 
                    image_list.append(x.get_attribute('src'))
            try:
                btn_next =  driver.find_element_by_class_name('_9zm2')
                btn_next.click()
            except NoSuchElementException:
                try:
                    next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                    next_post.click()
                except NoSuchElementException:
                    pass
                finally:
                    break
            except StaleElementReferenceException:
                try:
                    next_post = driver.find_element_by_class_name('_aaqg._aaqh')
                    next_post.click()
                except NoSuchElementException:
                    pass
                finally:
                    break
            finally:
                time.sleep(1)

        image_list = list(set(image_list))
        try:
            image_list = [requests.get(src).content for src in image_list]
        except ConnectionError:
            body.send_keys(Keys.ARROW_RIGHT)
            time.sleep(5)
            continue
        for src in image_list:
            if src not in downloaded_image_list:
                image_save(src, download_directory, upload_date)

                downloaded_image_list = downloaded_image_list + image_list
                time_delta = datetime.datetime.now().date() - datetime.date(yyyy, mm, dd)

        if time_delta > datetime.timedelta(days=90):
            break
    print()
    btn_close = driver.find_element_by_class_name('x78zum5.x6s0dn4.xl56j7k.xdt5ytf')
    btn_close.click()


NO KOREAN 250

NO KOREAN 249

NO KOREAN 248

NO KOREAN 247

NO FOLLOWING 246
yoonjujang Make Directory
92 days, 0:00:00

NO KOREAN 245
leojmakeup Make Directory


In [23]:
download_directory = os.path.join('./img', plusUrl)

In [24]:
download_directory

'./img\\nayoungkeem'